# Cleaning an .ods file in pandas

We are going to import some data from [Criminal Justice System statistics quarterly: March 2021](https://www.gov.uk/government/statistics/criminal-justice-system-statistics-quarterly-march-2021).

In [ ]:
import pandas as pd

In [ ]:
overview21mar = "https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/1011767/overview-tables-march-2021.ods"

## Dealing with the ods spreadsheet format

The first problem we have is that this isn't a CSV or XLS file.

How do we import an .ods file? [Off to Google](https://stackoverflow.com/questions/17834995/how-to-convert-opendocument-spreadsheets-to-a-pandas-dataframe):

> "This is available natively in pandas 0.25. So long as you have odfpy installed (conda install odfpy OR pip install odfpy) you can do
>
> `pd.read_excel("the_document.ods", engine="odf")`

First, then we need to install `odfpy`.


In [ ]:
!pip install odfpy

     |████████████████████████████████| 717 kB 40.4 MB/s 
  Created wheel for odfpy: filename=odfpy-1.4.1-py2.py3-none-any.whl size=160693 sha256=05068b1942b204368bb952a22a5fe63aefec65d5a6a914ee0461b44f22b47838
  Stored in directory: /root/.cache/pip/wheels/e2/f4/5d/a68c656235d33455a1d0f78e877acddfa006907a6d52d7e6ee
Successfully built odfpy


Once installed, we can use the `read_excel` function from `pandas`, but specify `engine="odf"`.

We can also specify which sheet we want, and get it to skip the first 4 rows before the heading row.

In [ ]:
overview21mardf = pd.read_excel(overview21mar, 
                                engine="odf", 
                                sheet_name="Q5_1", 
                                skiprows=4)

In [ ]:
overview21mardf.head()

,Offence type,Type of sentence,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,Indictable only offences,Total sentenced,19976,19330.0,17144.0,15681.0,14597.0,14050.0,13731.0,13263.0,12006.0,12718.0,10573
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,Immediate custody,13868,13664.0,12377.0,11450.0,10752.0,10393.0,10138.0,9762.0,8796.0,9023.0,6945
3,NaN,Suspended sentence,1699,1509.0,1517.0,1579.0,1683.0,1667.0,1539.0,1470.0,1155.0,1276.0,1414
4,NaN,Community sentence,3724,3589.0,2901.0,2254.0,1682.0,1472.0,1392.0,1382.0,1402.0,1804.0,1770


## Get rid of empty rows using `dropna`

Some rows are entirely empty. How can we get rid of those? Handily, `pandas` has a function that does that: `.dropna()`

However, if used without any arguments, this drops all rows with an `NaN` value or empty cell *anywhere* (note the index column loses all rows from 1-12):

In [ ]:
overview21mardf.dropna()

,Offence type,Type of sentence,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,Indictable only offences,Total sentenced,19976,19330.0,17144.0,15681.0,14597.0,14050.0,13731.0,13263.0,12006.0,12718.0,10573
13,Triable either way offences,Total sentenced,331344,318666.0,278796.0,277009.0,263120.0,244535.0,227700.0,202037.0,186242.0,184452.0,144916
26,Summary non-motoring,Total sentenced,495156,493826.0,456005.0,438850.0,463750.0,483856.0,470528.0,474189.0,460116.0,429662.0,188861
39,Summary motoring,Total sentenced,514207,462995.0,452644.0,450133.0,483453.0,510480.0,531615.0,510461.0,533451.0,531365.0,421399
52,Offence not known(7),Total sentenced,0,0.0,0.0,0.0,0.0,0.0,0.0,62.0,266.0,741.0,499
65,All offences,Total sentenced,1360683,1294817.0,1204589.0,1181673.0,1224920.0,1252921.0,1243574.0,1200012.0,1192081.0,1158938.0,766248
84,Offence type,Type of sentence,2011,2012.0,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0,2020.0,2021
85,Indictable only offences,Total sentenced,19976,19330.0,17143.0,15680.0,14594.0,14043.0,13728.0,13260.0,12006.0,12717.0,10571
98,Triable either way offences,Total sentenced,330513,317867.0,277989.0,276243.0,262318.0,243600.0,226862.0,201117.0,185318.0,183651.0,144565
111,Summary non-motoring,Total sentenced,493832,492718.0,455139.0,437869.0,462715.0,482200.0,468705.0,471679.0,457825.0,427706.0,188496


If we don't want that to happen we have to include an argument which specifies *which* columns we want to filter on.

In [ ]:
overview21mardf.dropna(subset=["2011"], inplace=True)

KeyError: ignored

## Dealing with `KeyError` due to numeric column headers

We hit an error here because the column has a numeric heading: `2011`. Really we shouldn't have numbers (integers) as column headings because that confuses pandas (which also uses integers as indices for columns).

So before we go any further we should fix that. First, here's the `columns` method which shows us the columns in a dataframe:

In [ ]:
overview21mardf.columns

Index([     'Offence type', 'Type of sentence ',                2011,
                      2012,                2013,                2014,
                      2015,                2016,                2017,
                      2018,                2019,                2020,
                      2021],
      dtype='object')

Note that the years don't have inverted commas around them: they aren't being treated as a string so couldn't be accessed that way.

There are a couple of ways to convert those to strings. The simplest is to create a list of the new column headings and assign them like so:

In [ ]:
newcolnames = ['Offence type', 'Type of sentence ', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021']
overview21mardf.columns = newcolnames

But that's quite laborious. Another way is to loop through the existing column names and convert them to strings like so:

In [ ]:
#create an empty list
newcolnames = []
#loop through the columns
for i in overview21mardf.columns:
  #convert to string
  strversion = str(i)
  #add to previously empty list
  newcolnames.append(strversion)
print(newcolnames)
#replace the old column names with the new ones
overview21mardf.columns = newcolnames

['Offence type', 'Type of sentence ', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021']


Another approach is to use [the `.map()` function](https://realpython.com/python-map-function/), which basically saves us having to loop.

This applies the specified function (`str` in this case) to each item in the list, in order to create a new one:

In [ ]:
overview21mardf.columns = overview21mardf.columns.map(str)
overview21mardf.columns

Index(['Offence type', 'Type of sentence ', '2011', '2012', '2013', '2014',
       '2015', '2016', '2017', '2018', '2019', '2020', '2021'],
      dtype='object')

Now the `dropna` function should work with that column name `'2011'` as a string:

In [ ]:
#try dropna again
overview21mardf.dropna(subset=['2011'], inplace=True)
#show results
overview21mardf

,Offence type,Type of sentence,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,Indictable only offences,Total sentenced,19976,19330.000,17144.0000,15681.0000,14597.0000,14050.0000,13731.0000,13263.0000,12006.0000,12718.0000,10573
2,NaN,Immediate custody,13868,13664.000,12377.0000,11450.0000,10752.0000,10393.0000,10138.0000,9762.0000,8796.0000,9023.0000,6945
3,NaN,Suspended sentence,1699,1509.000,1517.0000,1579.0000,1683.0000,1667.0000,1539.0000,1470.0000,1155.0000,1276.0000,1414
4,NaN,Community sentence,3724,3589.000,2901.0000,2254.0000,1682.0000,1472.0000,1392.0000,1382.0000,1402.0000,1804.0000,1770
5,NaN,Fine,45,21.000,23.0000,51.0000,29.0000,47.0000,95.0000,75.0000,72.0000,79.0000,84
...,...,...,...,...,...,...,...,...,...,...,...,...,...
156,NaN,Absolute discharge,8572,8013.000,7287.0000,6768.0000,5530.0000,8928.0000,4457.0000,5332.0000,4026.0000,4048.0000,1749
157,NaN,Conditional discharge,90457,85148.000,78064.0000,75049.0000,71312.0000,63623.0000,53718.0000,45476.0000,39848.0000,35803.0000,24198
158,NaN,Compensation,7790,6550.000,7940.0000,8487.0000,6180.0000,4886.0000,4870.0000,4955.0000,4850.0000,4506.0000,3539
159,NaN,Otherwise dealt with(5),24655,21997.000,16609.0000,18626.0000,19747.0000,11804.0000,13228.0000,12475.0000,11570.0000,11970.0000,8507


## Fill down using `ffill`

The `ffill` function will fill down when given `axis=0` (to fill across use `axis=1`).

Note that this fills row index 1 with the values from row 0, too. So it's a good thing we removed that row first.

In [ ]:
#fill down into empty cells (axis 0 means columns)
overview21mardf = overview21mardf.ffill(axis=0)
#show the results
overview21mardf

,Offence type,Type of sentence,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,Indictable only offences,Total sentenced,19976,19330.000,17144.0000,15681.0000,14597.0000,14050.0000,13731.0000,13263.0000,12006.0000,12718.0000,10573.0000
2,Indictable only offences,Immediate custody,13868,13664.000,12377.0000,11450.0000,10752.0000,10393.0000,10138.0000,9762.0000,8796.0000,9023.0000,6945.0000
3,Indictable only offences,Suspended sentence,1699,1509.000,1517.0000,1579.0000,1683.0000,1667.0000,1539.0000,1470.0000,1155.0000,1276.0000,1414.0000
4,Indictable only offences,Community sentence,3724,3589.000,2901.0000,2254.0000,1682.0000,1472.0000,1392.0000,1382.0000,1402.0000,1804.0000,1770.0000
5,Indictable only offences,Fine,45,21.000,23.0000,51.0000,29.0000,47.0000,95.0000,75.0000,72.0000,79.0000,84.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
156,All offences,Absolute discharge,8572,8013.000,7287.0000,6768.0000,5530.0000,8928.0000,4457.0000,5332.0000,4026.0000,4048.0000,1749.0000
157,All offences,Conditional discharge,90457,85148.000,78064.0000,75049.0000,71312.0000,63623.0000,53718.0000,45476.0000,39848.0000,35803.0000,24198.0000
158,All offences,Compensation,7790,6550.000,7940.0000,8487.0000,6180.0000,4886.0000,4870.0000,4955.0000,4850.0000,4506.0000,3539.0000
159,All offences,Otherwise dealt with(5),24655,21997.000,16609.0000,18626.0000,19747.0000,11804.0000,13228.0000,12475.0000,11570.0000,11970.0000,8507.0000


## Reshape wide to long

We have a column for each year but that's not ideal for data analysis. What we really want is a 'year' column.

To do this we need to [reshape the data from 'wide to long'](https://towardsdatascience.com/wide-to-long-data-how-and-when-to-use-pandas-melt-stack-and-wide-to-long-7c1e0f462a98). 

The `melt()` function is great for this. First, we need to know what the columns are called...

In [ ]:
#remind ourselves what the columns are
#we can also copy directly from this
overview21mardf.columns

Index(['Offence type', 'Type of sentence ', '2011', '2012', '2013', '2014',
       '2015', '2016', '2017', '2018', '2019', '2020', '2021'],
      dtype='object')

We use those in the ingredients of the `melt()` function: 

* `id_vars=` specifies which columns we want to keep as they were. This needs to be a list
* `value_vars` specifies which ones we want to reshape, using them as values instead of labels ('2011' is the value that will fill in a new column for the 'year')
* `var_name` specifies what we want to call this new column ('year')
* `value_name` specifies what we want to call the column which is going to contain all the values which were associated with the columns we're getting rid of. In this case it's 'total', indicating the total cases in each year and category.

In [ ]:
#use melt to convert the year column headings into a single column
overview21mardf.melt(id_vars=['Offence type', 'Type of sentence '], value_vars=['2011', '2012', '2013', '2014',
       '2015', '2016', '2017', '2018', '2019', '2020', '2021'], var_name='year', value_name='total')

,Offence type,Type of sentence,year,total
0,Indictable only offences,Total sentenced,2011,19976
1,Indictable only offences,Immediate custody,2011,13868
2,Indictable only offences,Suspended sentence,2011,1699
3,Indictable only offences,Community sentence,2011,3724
4,Indictable only offences,Fine,2011,45
...,...,...,...,...
1337,All offences,Absolute discharge,2021,1749
1338,All offences,Conditional discharge,2021,24198
1339,All offences,Compensation,2021,3539
1340,All offences,Otherwise dealt with(5),2021,8507


In [ ]:
#store those results permanently
overview21mardf = overview21mardf.melt(id_vars=['Offence type', 'Type of sentence '], value_vars=['2011', '2012', '2013', '2014',
       '2015', '2016', '2017', '2018', '2019', '2020', '2021'], var_name='year', value_name='total')

In [ ]:
overview21mardf.to_csv("cleaneddata.csv")

## Creating a pivot table

We can use that data to create a pivot table. Note that the results are presented in scientific notation.

In [ ]:
pivot12 = overview21mardf.pivot_table(index="Type of sentence ", 
                        values="total",
                        columns="year", 
                        aggfunc="sum")
pivot12

,year,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
Type of sentence,,,,,,,,,,,,
Absolute discharge,total,34374.0,3.210800e+04,2.924000e+04,2.711200e+04,22176.000,3.578000e+04,1.785000e+04,2.137400e+04,1.612800e+04,1.622600e+04,7.012000e+03
Average custodial sentence (months)(6),total,NaN,1.649658e+02,1.730099e+02,1.826475e+02,192.405,1.974078e+02,2.012571e+02,3.309403e+02,3.101586e+02,3.162768e+02,2.961874e+02
Community sentence,total,755320.0,6.965400e+05,5.654080e+05,4.960400e+05,444416.000,4.520040e+05,4.092280e+05,3.686600e+05,3.679260e+05,3.330000e+05,2.378520e+05
Compensation,total,31170.0,2.620200e+04,3.177400e+04,3.395000e+04,24724.000,1.955400e+04,1.948800e+04,1.983400e+04,1.941000e+04,1.803400e+04,1.416000e+04
Conditional discharge,total,361958.0,3.406680e+05,3.123440e+05,3.002700e+05,285340.000,2.545440e+05,2.149400e+05,1.819260e+05,1.594260e+05,1.432380e+05,9.680800e+04
Fine,total,3538968.0,3.369884e+06,3.243524e+06,3.207376e+06,3451232.000,3.593686e+06,3.658198e+06,3.596642e+06,3.673930e+06,3.598430e+06,2.280840e+06
Immediate custody,total,411348.0,4.218080e+05,3.805880e+05,3.724480e+05,363560.000,3.630080e+05,3.575240e+05,3.367240e+05,3.077360e+05,3.022360e+05,2.377440e+05
Otherwise dealt with(5),total,98650.0,8.803400e+04,6.647800e+04,7.455000e+04,79082.000,4.730200e+04,5.301800e+04,5.001600e+04,4.638800e+04,4.800600e+04,3.407600e+04
Suspended sentence,total,195272.0,1.905720e+05,1.762160e+05,2.019680e+05,215588.000,2.289520e+05,2.265680e+05,2.058140e+05,1.598140e+05,1.591380e+05,1.481020e+05


To fix that we just need to add `.astype()` to the end of the `pivot_table()` function to format any numbers in the results as integers - the text is ignored.

In [ ]:
pivot12 = overview21mardf[overview21mardf.year == '2012'].pivot_table(index="Type of sentence ", 
                        values="total",
                        columns="year", 
                        aggfunc="sum").astype(int)
pivot12

year,2012
Type of sentence,
Absolute discharge,32108
Average custodial sentence (months)(6),164
Community sentence,696540
Compensation,26202
Conditional discharge,340668
Fine,3369884
Immediate custody,421808
Otherwise dealt with(5),88034
Suspended sentence,190572


In [ ]:
pivot12 = overview21mardf[overview21mardf.year == '2011'].pivot_table(index="Type of sentence ", 
                        values="total",
                        columns="year", 
                        aggfunc="sum").astype(int)
pivot12

,year,2011
Type of sentence,,
Absolute discharge,total,34374
Community sentence,total,755320
Compensation,total,31170
Conditional discharge,total,361958
Fine,total,3538968
Immediate custody,total,411348
Otherwise dealt with(5),total,98650
Suspended sentence,total,195272
Total sentenced,total,5427060


## Using SQL-style functions `where`, `isin` etc.

We can also use SQL-style functions to filter a dataframe [such as `isin`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.isin.html).